In [36]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install evaluate

In [39]:
pip install openai==0.28

GPT-4 ranking. The output of MindMap only choose the summary result (Output1)

In [ ]:
import openai
import csv
from time import sleep

input_file = 'D:\浏览器\MindMap-main\MindMap-main\output.csv'
output_file = 'output_ranking.csv'

openai.api_key = 

def prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text):
  prompt_template = """
  Reference: {reference}

  output1: {output1}

  output2: {output2}

  output3: {output3}

  output4: {output4}

  output5: {output5}

  output6: {output6}

  According to the facts of the disease and the drug and test recommendations in reference output, order the fact match of the output from highest to lowest.

  The output followed the format bellow：

  1. Output1
  2. Output5
  3. Output2
  4. Output3
  5. Output4
  6. Output6
  """

  prompt = prompt_template.format(reference=reference_text, output1=output1_text, output2=output2_text, output3=output3_text, output4=output4_text, output5=output5_text, output6=output6_text)
  response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[
          {"role": "system", "content": "You are a excellent."},
          {"role": "user", "content": prompt}
      ]
  )

  generated_text = response.choices[0].message.content
  return generated_text

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.append("gpt4_ranking")
  print("[Header] 列名列表:", header)
  writer.writerow(header)

  for row in reader:
    #打印列数
    print("[Row] 列数:", len(row))
    #打印每一列的名称
    print("[Header] 列名列表:", header)

    reference_text = [row[1].strip("\n")]
    output1_text = [row[2].strip("\n")]
    output2_text = [row[3].strip("\n")]
    output3_text = [row[4].strip("\n")]
    output4_text = [row[5].strip("\n")]
    output5_text = [row[6].strip("\n")]
    output6_text = [row[7].strip("\n")]

    flag = 0
    while flag == 0:
      try:
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text)
        flag = 1
      except:
        sleep(40)
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text,output4_text,output5_text,output6_text)

    print(str1)
    print('\n')

    row.append(str1)
    writer.writerow(row)

[Header] 列名列表: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'GPT4', 'gpt4_ranking']
[Row] 列数: 8
[Header] 列名列表: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'GPT4', 'gpt4_ranking']
1. Output1
2. Output5
3. Output4
4. Output3
5. Output6
6. Output2


[Row] 列数: 8
[Header] 列名列表: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'GPT4', 'gpt4_ranking']
1. Output2
2. Output6
3. Output1
4. Output5
5. Output4
6. Output3


[Row] 列数: 8
[Header] 列名列表: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'GPT4', 'gpt4_ranking']
1. Output5
2. Output1
3. Output4
4. Output3
5. Output2
6. Output6


[Row] 列数: 8
[Header] 列名列表: ['Question', 'Label', 'MindMap', 'GPT3.5', 'BM25_retrieval', 'Embedding_retrieval', 'KG_retrieval', 'GPT4', 'gpt4_ranking']
1. Output2
2. Output4
3. Output5
4. Output6
5. Output

In [41]:
import re
import csv

input_file = 'output_ranking.csv'
output_file = 'output_ranking_compute.csv'

with open(input_file, 'r', newline="", encoding='cp1252') as f_input, open(output_file, 'w', newline='', encoding='utf-8') as f_output:
    reader = csv.reader(f_input)
    writer = csv.writer(f_output)

    header = next(reader)
    header.extend(["output1_ranking", "output2_ranking", "output3_ranking", 
                  "output4_ranking", "output5_ranking", "output6_ranking"])
    writer.writerow(header)

    row_count = 0
    for row in reader:
        try:
            row_count += 1
            # 使用第11列（索引为10）作为排名信息
            output_text = row[8]
            print(f"\n处理第{row_count}行: {output_text}")

            # 提取 "1. Output2 2. Output4" 这样的格式
            matches = re.findall(r'(\d+)\.\s*Output(\d+)', output_text)
            
            if not matches:
                print(f"警告：无法从文本中提取排名: {output_text}")
                row.extend([0, 0, 0, 0, 0, 0])
                writer.writerow(row)
                continue
                
            print(f"匹配结果: {matches}")
            
            # 创建一个字典，键为Output编号，值为其排名
            output_rankings = {}
            for rank_str, output_num in matches:
                rank = int(rank_str)
                output_num = int(output_num)
                output_rankings[output_num] = rank
                print(f"Output{output_num}的排名是第{rank}名")
            
            # 准备添加到行中的排名数据
            rankings_to_add = []
            for i in range(1, 7):
                rank = output_rankings.get(i, 0)  # 如果没有找到排名，默认为0
                rankings_to_add.append(rank)
            
            print(f"添加的排名: {rankings_to_add}")
            
            # 将排名添加到行中
            row.extend(rankings_to_add)
            writer.writerow(row)
            
        except Exception as e:
            print(f"处理行时出错: {e}")
            # 出错时添加默认值
            row.extend([0, 0, 0, 0, 0, 0])
            writer.writerow(row)

print("处理完成！")


处理第1行: 1. Output4
2. Output5
3. Output1
4. Output6
5. Output3
6. Output2
匹配结果: [('1', '4'), ('2', '5'), ('3', '1'), ('4', '6'), ('5', '3'), ('6', '2')]
Output4的排名是第1名
Output5的排名是第2名
Output1的排名是第3名
Output6的排名是第4名
Output3的排名是第5名
Output2的排名是第6名
添加的排名: [3, 6, 5, 1, 2, 4]

处理第2行: 1. Output 3
2. Output 1
3. Output 6
4. Output 2
5. Output 5
6. Output 4
警告：无法从文本中提取排名: 1. Output 3
2. Output 1
3. Output 6
4. Output 2
5. Output 5
6. Output 4

处理第3行: 1. Output6
2. Output4
3. Output5
4. Output1
5. Output3
6. Output2
匹配结果: [('1', '6'), ('2', '4'), ('3', '5'), ('4', '1'), ('5', '3'), ('6', '2')]
Output6的排名是第1名
Output4的排名是第2名
Output5的排名是第3名
Output1的排名是第4名
Output3的排名是第5名
Output2的排名是第6名
添加的排名: [4, 6, 5, 2, 3, 1]

处理第4行: 1. Output3
2. Output6
3. Output5
4. Output2
5. Output4
6. Output1
匹配结果: [('1', '3'), ('2', '6'), ('3', '5'), ('4', '2'), ('5', '4'), ('6', '1')]
Output3的排名是第1名
Output6的排名是第2名
Output5的排名是第3名
Output2的排名是第4名
Output4的排名是第5名
Output1的排名是第6名
添加的排名: [6, 4, 1, 5, 3, 2]

处理第5行: 1. Output3
2. Outp

In [1]:
import pandas as pd

def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path, encoding='utf-8')
    average = df[column_name].mean()
    return average

column =["output1_ranking","output2_ranking","output3_ranking","output4_ranking","output5_ranking","output6_ranking"]

file_path = './output_ranking_compute.csv'
for column_name in column:
  average = calculate_column_average(file_path, column_name)
  print(f"The average of column {column_name} is: {average}")




The average of column output1_ranking is: 2.5609756097560976
The average of column output2_ranking is: 3.7560975609756095
The average of column output3_ranking is: 3.097560975609756
The average of column output4_ranking is: 2.951219512195122
The average of column output5_ranking is: 2.5853658536585367
The average of column output6_ranking is: 3.4878048780487805
